In [4]:
import yfinance as yf
import pandas as pd

def analizar_proximidad_bollinger(ticker, periodo=20, desviacion=2, margen_porcentaje=0.10):
    try:
        df = yf.download(f"{ticker.upper()}-USD", period="1d", interval="1m", progress=False)
        if df.empty: return None
        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.get_level_values(0)
        df.columns = [c.lower() for c in df.columns]
        
        sma = df['close'].rolling(window=periodo).mean().iloc[-1]
        std = df['close'].rolling(window=periodo).std().iloc[-1]
        precio = df['close'].iloc[-1]
        
        niveles = {"sup1": sma + std, "m": sma, "inf1": sma - std, "inf2": sma - (desviacion * std)}
        rango_total = niveles['sup1'] - niveles['inf2']
        umbral = rango_total * margen_porcentaje 
        
        banda_cercana = None
        for nombre, valor_banda in niveles.items():
            if abs(precio - valor_banda) <= umbral:
                banda_cercana = nombre
                break
        return {"banda_cercana": banda_cercana, "niveles": niveles, "precio": precio}
    except Exception as e:
        print(f"❌ Error Bollinger ({ticker}): {e}")
        return None